<a href="https://colab.research.google.com/github/submouse9903/uos-deepLearning/blob/main/U47768_CH05_LSTM(%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_Modeling).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN을 이용한 텍스트 분류(Text Classification)

## 1) 데이터 전처리 

### - 데이터
: 이 실습에서 사용하는 데이터는 https://github.com/lih0905/korean-pytorch-sentiment-analysis 에 있는 영화 평점 데이터를 활용하였다.

In [ ]:
!pip install konlpy
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torchtext
from konlpy.tag import Okt
import collections
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 20.5 MB/s eta 0:00:00


In [ ]:
okt=Okt()  

In [ ]:
train_data = pd.read_csv("http://ranking.uos.ac.kr/class/RB/rating_train.csv")
test_data = pd.read_csv("http://ranking.uos.ac.kr/class/RB/rating_test.csv")

In [ ]:
train_data.head()

,id,text,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
train_data = train_data[0:1000]

In [ ]:
from tqdm import tqdm 
morph_counter = collections.defaultdict(int)
text_line = train_data["text"]
for text in tqdm(text_line):
  for m in okt.morphs(text):
    morph_counter[m] += 1

100%|██████████| 1000/1000 [00:17<00:00, 57.73it/s]


In [ ]:
morph_to_id = {'[PAD]': 0, '[UNK]': 1}
for w, cnt in tqdm(morph_counter.items()):
    morph_to_id[w] = len(morph_to_id)
print(len(morph_to_id))

100%|██████████| 4770/4770 [00:00<00:00, 557091.58it/s]

4772


In [ ]:
id_to_morph = {}
for w, idx in tqdm(morph_to_id.items()):
  id_to_morph[idx] = w
print(len(id_to_morph))  

100%|██████████| 4772/4772 [00:00<00:00, 813461.44it/s]

4772


In [ ]:
# pre-padding
def textEncoding(text_list, max_length, morph_to_id):
  m =  max_length-len(text_list)
  if m>=0:
    text_list_id = [0 if i<m else morph_to_id.get(text_list[i-m],1) for i in range(max_length)]
  # 입력 text 가 최대 길이보다 긴 경우
  else:
    text_list_id = [morph_to_id.get(text_list[i],1) for i in range(max_length)]
  return text_list_id

In [ ]:
max_length = 10
X = []
for text in tqdm(train_data["text"]):
  text_list = okt.morphs(text)
  X.append(textEncoding(text_list, max_length , morph_to_id))

100%|██████████| 1000/1000 [00:04<00:00, 223.95it/s]


In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
X_t = torch.tensor(X).long()
Y_t = torch.tensor(train_data["label"]).long()
tr_data = TensorDataset(X_t, Y_t)
train_dataloader = DataLoader(tr_data, batch_size=5, shuffle=True)

In [ ]:
#Y_t

In [ ]:
embedding = torch.nn.Embedding(len(morph_to_id), 40, padding_idx = 0)

In [ ]:
for x, y in train_dataloader:
  z = embedding(x)
  print("shape of embedding vector:", z.shape)
  break

shape of embedding vector: torch.Size([5, 10, 40])


In [ ]:
y

tensor([0, 1, 0, 1, 0])

## 2) 모델링

### - 모형

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(1)
if device == 'cuda':
    torch.cuda.manual_seed_all(1)

### Parameters:
- input_size – The number of expected features in the input x

- hidden_size – The number of features in the hidden state h

- num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1

- bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True

- batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False

- dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0

- bidirectional – If True, becomes a bidirectional LSTM. Default: False

- proj_size – If > 0, will use LSTM with projections of corresponding size. Default: 0


In [ ]:
# 10차원 벡터가 입력
# 20차원 데이터가 hidden vector 
# 2개 층으로 구성
rnn = nn.LSTM(input_size = 10, hidden_size = 20, num_layers = 2, 
              batch_first = True)

### Input:
- Batch_first = False 인 경우에 (seq, batch, feature) 로 input data x 를 입력한다. (Default 값으로 지정되어 있다)
- Batch_first = True 인 경우에 (batch, seq, feature) 로 input data x 를 입력한다. 
- 출력값 역시 Batch_first 옵션을 따른다.

In [ ]:
# sample 3개, 길이 5, feature 20
input = torch.randn(3, 5, 10)

### h0 
- Batch_first = True 와 상관없이 (num_layers*D, batch, hidden_size) 로 h0 를 입력한다.  (만약 bidirectional 이면 D = 2, 아니면 D = 1)
- 위의 예제에서는 2개 층, 3개 샘플,  20차원이므로 (2,3,20)를 입력한다.




In [ ]:
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [ ]:
print(hn.shape)
print(cn.shape)

torch.Size([2, 3, 20])
torch.Size([2, 3, 20])


In [ ]:
output.shape

torch.Size([3, 5, 20])

In [ ]:
# Define the RNN model
class myLSTM(nn.Module):
    def __init__(self, size_vocab, input_size, hidden_size, num_layers):
        super(myLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(size_vocab, hidden_size, padding_idx = 0)
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(nn.Linear(hidden_size, 2),
                                nn.Softmax(dim=1))
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x = self.embedding(x).to(device)
        out, _ = self.rnn(x, (h0,c0))
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [ ]:
for x, y in train_dataloader:
  z = embedding(x)
  print("shape of embedding vector:", z.shape)
  break

shape of embedding vector: torch.Size([5, 10, 40])


In [ ]:
size_vocab = len(morph_to_id)
input_size = max_length
hidden_size = 10
num_layer = 1

In [ ]:
model = myLSTM(size_vocab, input_size, hidden_size, num_layer)
for x, y in train_dataloader:
  z = model(x)
  print("shape of embedding vector:", z.shape)
  break

shape of embedding vector: torch.Size([5, 2])


In [ ]:
learning_rate = 0.005
model = myLSTM(size_vocab, input_size, hidden_size, num_layer).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.zero_grad()


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  for inputs, labels in tqdm(train_dataloader):
    input = inputs.to(device)
    label = labels.to(device)
    output = model(input)
    loss = criterion(output, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(loss)

100%|██████████| 200/200 [00:00<00:00, 333.62it/s]


tensor(0.7103, grad_fn=<NllLossBackward0>)


100%|██████████| 200/200 [00:00<00:00, 307.49it/s]


tensor(0.7604, grad_fn=<NllLossBackward0>)


100%|██████████| 200/200 [00:00<00:00, 451.20it/s]


...

tensor(0.5143, grad_fn=<NllLossBackward0>)


100%|██████████| 200/200 [00:00<00:00, 455.69it/s]


tensor(0.3210, grad_fn=<NllLossBackward0>)


100%|██████████| 200/200 [00:00<00:00, 327.30it/s]

tensor(0.3146, grad_fn=<NllLossBackward0>)


In [ ]:
model(input)

tensor([[5.9117e-03, 9.9409e-01],
        [3.0324e-05, 9.9997e-01],
        [9.9983e-01, 1.7464e-04],
        [6.0048e-03, 9.9400e-01],
        [1.7696e-05, 9.9998e-01]], grad_fn=<SoftmaxBackward0>)